In [ ]:
pip install shioaji > log.txt

# 登入帳戶

In [ ]:
import shioaji as sj

api = sj.Shioaji()
accounts = api.login("user_id", "password")


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


# 匯入憑證

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!ls

sample_data  shioaji.log  Sinopac.pfx


In [ ]:
api.activate_ca(
    ca_path="Sinopac.pfx",
    ca_passwd="user_id",
    person_id="user_id",
)

Ca Initial Done.


0

# 取得憑證

In [ ]:
ticks = api.ticks(api.Contracts.Stocks["2330"], "2019-03-04")

import pandas as pd
df = pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)
df.head()

,bid_volume,ask_price,bid_price,ask_volume,close,ts,volume
0,0,0,0,0,239.5,2019-03-04 09:00:00.842,5082
1,0,0,0,0,239.5,2019-03-04 09:00:05.873,54
2,0,0,0,0,239.5,2019-03-04 09:00:10.901,11
3,0,0,0,0,239.5,2019-03-04 09:00:15.926,32
4,0,0,0,0,239.5,2019-03-04 09:00:20.949,8


# 股票倉位調整函式

In [ ]:
import pandas as pd

def create_new_orders(target_positions):
  """create new orders in order to rebalance the old positions to new positions
    
    Parameters:
    target_position (dict): a dictionary with stock_id and the number of lot
      
      example: 

        target_positions = {
          '2414': 4.0,
          '2430': 6.0,
          '2616': 3.0,
        }

    Returns:
    dict: new orders which update old positions.

      example: if your original positions are {'2414': 2, '2330': 1} then with the target_position example above,
               the following return is obtained.

        {
          '2414': 2.0,
          '2330': -1
          '2430': 6.0,
          '2616': 3.0,
        }

  """

  # get present positions
  positions = api.list_positions(api.stock_account)
  present_positions = {p.code: p.quantity for p in positions}
  target_positions = pd.Series(target_positions).astype(int).to_dict()
  # print present and target positions
  print('Present positions:')
  print(pd.Series(present_positions))
  print('------------------')
  print('Target positions:')
  print(pd.Series(target_positions))
  print('------------------')

  # calculate the difference between present position and target position
  all_codes = set(list(target_positions.keys()) + list(present_positions.keys()))
  new_orders = (pd.Series(target_positions).reindex(all_codes).fillna(0) - 
                pd.Series(present_positions).reindex(all_codes).fillna(0)).astype(int)
  new_orders = new_orders[new_orders!=0].to_dict()

  # print the new orders
  print('new orders to rebalance:')
  if new_orders:
    print(pd.Series(new_orders))
  else:
    print('None')
  print('------------------')
  return new_orders

def cancel_all_unfilled_orders():
  """Cancel all unfilled orders"""

  api.update_status(api.stock_account)
  for trade in api.list_trades():
    if trade.status.status in ['Cancelled', 'Failed', 'Filled']: # Failed PreSubmitted
      pass
    else:
      print('operation: cancel trade\t', trade)
      api.cancel_order(trade)

def execute_orders(new_orders):
  
  # query market data
  contracts = [api.Contracts.Stocks.get(code) for code in new_orders.keys()]
  market_data = api.snapshots(contracts)
  market_data = {snapshot.code: snapshot for snapshot in market_data}

  # cancel unrealized orders
  cancel_all_unfilled_orders()

  # make orders
  for code, quantity in new_orders.items():
    contract = api.Contracts.Stocks.get(code)
    price = market_data[code].close
    
    action = 'Buy' if quantity > 0 else 'Sell'

    if quantity == 0:
      continue

    order = api.Order(price=price,
                    quantity=abs(int(quantity)), 
                    action=action,
                    price_type="LMT",
                    order_type="ROD",
                    order_lot="Common",
                    account=api.stock_account
    )
    trade = api.place_order(contract, order)
    print('operation: new trade:\t', trade)

# 產生委託單
上方已經寫好函式

方便大家直接使用

以下是介紹：

.

首先

`create_new_orders(target_position)`

幫你計算「目標倉位 `target_position`」跟「實際倉位」的股票數量差異

然後幫你產生出需要「委託的股票清單」，

.

此時新的委託單 `new_orders` **不會直接委託**！

而讓你確認 `new_orders` 是你想要的委託清單後

再用

`execute_orders(new_orders) `

來下委託單
委託單皆會以當前價格「限價」掛單


In [ ]:
target_positions = {'2414': 4,
 '2430': 6,
 '2616': 3,
 '3029': 3,
 '3032': 3,
 '3056': 3,
 '3325': 5,
 '3537': 2,
 '5609': 3,
 '6126': 5
}

new_orders = create_new_orders(target_positions)
new_orders

Present positions:
2414    4
2430    6
2616    3
3029    3
3032    3
3056    3
3325    5
3537    2
5609    3
6126    6
dtype: int64
------------------
Target positions:
2414    4
2430    6
2616    3
3029    3
3032    3
3056    3
3325    5
3537    2
5609    3
6126    5
dtype: int64
------------------
new orders to rebalance:
6126   -1
dtype: int64
------------------


{'6126': -1}

In [ ]:
new_orders

{'6126': -1}

In [ ]:
execute_orders(new_orders)

In [ ]:
api.list_positions(api.stock_account)

[Position(code='2414', direction=<Action.Buy: 'Buy'>, quantity=4, price=24.18, pnl=-563.0, yd_quantity=4),
 Position(code='2430', direction=<Action.Buy: 'Buy'>, quantity=6, price=17.78, pnl=-24.0, yd_quantity=6),
 Position(code='2616', direction=<Action.Buy: 'Buy'>, quantity=3, price=30.74, pnl=-239.0, yd_quantity=3),
 Position(code='3029', direction=<Action.Buy: 'Buy'>, quantity=3, price=34.75, pnl=-458.0, yd_quantity=3),
 Position(code='3032', direction=<Action.Buy: 'Buy'>, quantity=3, price=30.14, pnl=-79.0, yd_quantity=3),
 Position(code='3056', direction=<Action.Buy: 'Buy'>, quantity=3, price=34.5, pnl=293.0, yd_quantity=3),
 Position(code='3325', direction=<Action.Buy: 'Buy'>, quantity=5, price=18.68, pnl=-543.0, yd_quantity=5),
 Position(code='3537', direction=<Action.Buy: 'Buy'>, quantity=2, price=41.56, pnl=-485.0, yd_quantity=2),
 Position(code='5609', direction=<Action.Buy: 'Buy'>, quantity=3, price=32.6, pnl=-3706.0, yd_quantity=3),
 Position(code='6126', direction=<Action.

In [ ]:
api.list_profit_loss(api.stock_account,'2020-05-05','2020-05-30')

[]